<a href="https://colab.research.google.com/github/G-Conard/ds595-final-project/blob/main/Cross_validation_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Keras Tuner
!pip install keras-tuner

# Import necessary libraries
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras_tuner import Hyperband, Objective

# Load and prepare data
data = pd.read_csv('/content/weather_stations_data.csv')
data = data.dropna()
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp', inplace=True)
data = data[['heat_index']].values.astype('float32')

scaler = MinMaxScaler(feature_range=(-1, 1))
sc = scaler.fit_transform(data)

timestep = 288
X, Y = [], []
for i in range(len(sc) - timestep):
    X.append(sc[i:(i + timestep)])
    Y.append(sc[i + timestep])
X, Y = np.array(X), np.array(Y)
X = X[:, :, None]

split_idx = int(0.8 * len(X))
X_train, Y_train = X[:split_idx], Y[:split_idx]
X_test, Y_test = X[split_idx:], Y[split_idx:]

def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Choice('units', values=[32, 64]),
                   activation=hp.Choice('activation', values=['relu', 'tanh']),
                   input_shape=(timestep, 1),
                   return_sequences=True))
    model.add(Dropout(rate=hp.Choice('dropout', values=[0.1, 0.2])))
    model.add(LSTM(units=hp.Choice('units', values=[32, 64]),
                   activation=hp.Choice('activation', values=['relu', 'tanh']),
                   return_sequences=False))
    model.add(Dense(1))
    model.compile(optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),  # Fixed to one type for simplicity
                  loss='mse',
                  metrics=['mse'])
    return model

# Set up the tuner
tuner = Hyperband(
    build_model,
    objective=Objective("val_mse", direction="min"),
    max_epochs=20,  # Adjusted inside the search
    executions_per_trial=1,
    directory='my_dir',
    project_name='weather_prediction')

# Perform the search with limited epoch choices
for epochs in [10, 20]:  # Limited to two epoch values
    for batch_size in [32, 64]:  # Limited to two batch sizes
        tuner.search(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=1)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Optionally, evaluate the model
loss, mse = best_model.evaluate(X_test, Y_test)
print("Test MSE:", mse)

# Print a summary of all trials
tuner.results_summary()


Trial 22 Complete [00h 04m 26s]
val_mse: 0.0056853159330785275

Best val_mse So Far: 0.0024269085843116045
Total elapsed time: 01h 18m 51s
42/42 [==============================] - 6s 93ms/step - loss: 0.0012 - mse: 0.0012
Test MSE: 0.0011869187001138926
Results summary
Results in my_dir/weather_prediction
Showing 10 best trials
Objective(name="val_mse", direction="min")

Trial 0016 summary
Hyperparameters:
units: 64
activation: tanh
dropout: 0.1
optimizer: adam
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0015
Score: 0.0024269085843116045

Trial 0017 summary
Hyperparameters:
units: 64
activation: tanh
dropout: 0.1
optimizer: rmsprop
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0013
Score: 0.0025877903681248426

Trial 0015 summary
Hyperparameters:
units: 64
activation: tanh
dropout: 0.1
optimizer: adam
tuner/epochs: 7
tuner/initial_epoch: 3
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 0009
Score: 0.